In [1]:
# 신경망 학습 
# 학습 : 훈련 데이터로부터 가중치 매개변수의 최적값을 자동으로 획득하는 것
# 손실함수 : 신경망이 학습할 수 있도록 해주는 지표이다. 이 손실 함수의 결과값을 가장 작게 만드는 가중치 매개변수를 찾는 것이 학습의 목표.

'''
2장에서의 퍼셉트론도 직선으로 분리할 수 있는(선형 분리 가능) 문제라면 데이터로 자동학습할 수 있다.
선형 분리 가능 문제는 유한번의 학습을 통해 풀 수 있다는 사실이 [퍼셉트론 수렴 정리]로 증명되었다.
그러나 비선형 분리 문제는 자동으로 학습할 수 없다.
'''

In [ ]:
# 이번 챕터에서는, 지난 챕터에서 수작업으로 입력했던 가중치 매개변수를 데이터 학습을 통해 자동으로 결정하는 '신경망 학습'을 공부한다.

# 기계학습에서는 ,어떤 문제를 풀 때 사람의 개입(직관과 경험)을 최소화하고 수집한 데이터로부터 패턴을 찾으려 시도한다.
# 규칙을 '사람'이 만드는 방식에서 '기계'가 데이터를 배우는 방식으로의 패러다임 전환
'''
1. 데이터셋 -> 사람이 생각한 알고리즘 : 사람 -> 결과
2. 데이터셋 -> 사람이 생각한 특징(SIFT, HOG등) : 사람 -> 기계학습(SVM,KNN 등) : 기계 -> 결과
3. 데이터셋 -> 신경망(딥러닝) : 기계 -> 결과

1번과 2번에서는 사람이 개입할 여지(알고리즘, 특징 추출)가 있었지만, 3번에서는 사람의 개입을 원천 배제하여 오직 기계가 스스로 학습하기만 한다.
이러한 특징 때문에 딥러닝을 [종단간 기계학습]이라고도 부른다. '처음부터 끝까지 사람의 개입없이 데이터에서 결과를 출력한다'는 뜻이다.
'''
# 신경망의 이점은 모든 문제를 같은 맥락에서 풀 수 있다는 것이다. 숫자를 인식하든, 개를 인식하든, 사람을 인식하든 세부사항과 관련없이 신경망은 주어진 데이터를 온전히 학습하고, 주어진 문제의 패턴을 발견하려 시도한다.
# 즉, 모든 문제를 주어진 데이터 그대로 입력데이터로 활용해 end-to-end로 학습할 수 있다.

In [3]:
# 기계학습에서 데이터를 취급할 때, 데이터를 두가지로 나누는 것이 일반적이다.
'''
훈련 데이터training data : 최적의 매개변수를 찾도록 학습하는 데이터
시험 데이터test data : 앞서 훈련한 모델(매개변수)의 실력을 평가하는 데이터
'''

# 이렇게 두가지로 데이터를 나누는 이유는 범용적으로 사용할 수 있는 모델을 찾기 위함이다.
# 하나의 데이터 셋에만 지나치게 최적화된 상태를 '오버피팅'이라고 부르며, 오버피팅을 피하는 것은 범용능력을 갖추려는 기계학습의 중요한 과제이다.

In [ ]:
# 신경망 학습에서는 현재의 상태를 '하나의 지표'로 표현하며, 해당 지표를 가장 좋게 만들어주는(최적의) 가중치 매개변수의 값을 탐색한다.
# 이러한 지표를 '손실 함수'라고 부르며, 임의의 함수를 사용할 수도 있지만 일반적으로는 오차제곱합과 교차 엔트로피 오차를 사용한다.
'''
손실 함수는 신경망 성능의 '나쁨'을 나타내는 지표로, 현재 신경망이 훈련데이터를 얼마나 잘 처리하지 '못'하느냐를 의미한다.
결국 이 값을 최소화 하는것이 곧 성능을 '나쁘지 않게', 즉 좋게 만드는 것이다.
'''

In [4]:
# 손실 함수의 종류
# 오차제곱합sum of squares for error,SSE
# E = 1/2*Σ(출력(추정)값-정답레이블)^2

import numpy as np

def sum_squares_error(y, t): # y와 t는 넘파이 배열, y:신경망 출력값, t:정답 레이블
    return 0.5 * np.sum((y-t)**2)

# 정답은 '2', one-hot encoding 방법
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

# 예1 : '2'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(sum_squares_error(np.array(y),np.array(t)))

# 예2 : '7'일 확률이 가장 높다고 추정함(0.6)
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(sum_squares_error(np.array(y),np.array(t)))

# 오차제곱합 기준으로 첫 번째 추정결과가 더 오차가 작으니, 정답에 더 가까울 것으로 판단할 수 있다.

0.09750000000000003
0.5975


In [5]:
# 교차 엔트로피 오차cross entropy error, CEE
# E = -Σ(정답레이블) * log(출력(추정)값)

def cross_entropy_error(y, t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta)) # np.log의 인수에 0을 입력하면 -inf가 되어 계산 진행이 불가능하므로, 아주 작은 값(delta)를 더해서 y + delta가 0이 되는것을 막아준다.

# 정답은 '2', one-hot encoding 방법
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
print(cross_entropy_error(np.array(y), np.array(t)))

# 교차 엔트로피 오차 기준으로 첫 번째 추정 결과가 더 오차가 작으니, 정답에 더 가까울 것으로 판단할 수 있다.

0.510825457099338
2.302584092994546


In [6]:
# 지금까지는 1개의 데이터를 이용하여 손실 함수 값을 구했지만, 실제로는 n개의 데이터를 이용해야한다.
# 1개씩의 손실 함수 값을 총 n개 구하여 모두 합친 뒤, n으로 나누어 평균 손실 함수 값을 뽑아낸다.

# 그렇지만, 수만~수천만에 해당하는 합을 모두 구하려면 시간이 걸리므로, 랜덤한 일부를 추려 전체의 '근사치'로 이용할 수 있다.
# 이 일부를 '미니배치mini-batch'라고 하며, 이 미니배치를 이용한 학습 방식을 미니배치 학습이라고 한다.

In [9]:
# MNIST 데이터셋을 미니배치학습

import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = \
    load_mnist(normalize=True, one_hot_label=True)

print(x_train.shape) # 훈련데이터는 60000개, 입력 데이터는 784열(28*28 이미지)
print(t_train.shape) # 정답레이블은 60000개, 10줄짜리 데이터

(60000, 784)
(60000, 10)


In [10]:
train_size = x_train.shape[0] # 훈련 데이터의 갯수(60000개)
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size) # train_size(60000)에서 무작위 batch_size(10)개 추출
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]

In [12]:
# 배치 데이터를 지원하는 교차 엔트로피 오차 구현

# 정답 레이블이 원-핫 인코딩일 때
def cross_entropy_error(y, t): # y:신경망 출력, t:정답 레이블
    if y.ndim == 1: # 신경망 출력이 1차원이라면 == 데이터 하나당 교차 엔트로피 오차를 구하는 경우
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_szie = y.shape[0]
    return -np.sum(t * np.log(y + 1e-7)) / batch_size # batch_size로 정규화

# 정답 레이블이 원-핫 인코딩이 아니라 '2', '7'등의 숫자 레이블로 주어졌을 때
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    batch_szie = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
    # 원-핫 인코딩일 때 t가 0인 원소는 교차 엔트로피 오차도 0이므로 해당 계산 무시. 따라서 이런 식으로 바뀐다
    # np.arange(batch_size)는 0부터 batch_size -1까지의 numpy 배열 생성
    # y[np.arange(batch_size),t]는 각 데이터의 정답 레이블에 해당하는 신경망 출력을 추출 ex - [y[인덱스,해당 인덱스 데이터의 정답], ...]

In [ ]:
# 왜 '정확도' 대신 '손실 함수'를 지표로 삼는가? --> '미분' 값에 주목하라.
'''
100장중 32장을 올바르게 인식했을 때, 정확도는 32%이다.
가중치 매개변수를 조금 바꾸어도 32%이다가, 어느순간 33%로 바뀐다.
따라서 정확도 값은 가중치 매개변수의 차이에 정확히 반응하지 않고 불연속적이다. (미분값이 높은 빈도로 0이 된다)

그러나 손실 함수를 지표로 삼는다면, 0.92543... 같은 수치들이 연속적으로 변화하여, 작은 가중치 매개변수의 변화에도 반응한다.

이러한 이유는 활성화 함수로 계단함수를 사용하지 않는 이유와도 일맥상통한다.
따라서 계단 함수를 이용하면 손실 함수를 지표로 삼는 것이 아무런 의미가 없게 된다.
매개변수의 작은 변화가 주는 파장을 계단함수가 무시하여 손실 함수의 값에는 아무런 변화가 나타나지 않기 때문이다.
'''